# Basic Portfolio Analysis

#### Notebook: Data Scraping/Collection from the COL Financial Platform


In [6]:
# standard python libraries
import json
import datetime
import pandas as pd
import numpy as np

# library for getting historical stock price data
import fastquant as fq

# library for automating web scraping
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

# Part 1: Collecting the Data

##### For this section, we will use the `Selenium` to log-in to the broker's platform. Selenium automates brower nagivation, which we can use for data scraping. For safety, my credentials are stored in a separate file, which the program would read.

In [7]:
# For security, we do not hard code the credentials, instead we use a separate txt file for it.
with open('../credentials/credentials_mainfund.txt') as f:
# with open('credentials_thirdtier.txt') as f:
    data = f.read()
    credentials = json.loads(data)

In [8]:
# So that the browser does not appear, we will use the --headless and --disable-gpu parameters

options = Options()
options.add_argument('--headless') # headless so the chrome tab will not pop up - faster runtime
options.add_argument('--disable-gpu')  # disable gpu required for headless selenium testing

driver = webdriver.Chrome(
    'driver/chromedriver_mac_arm64/chromedriver', 
    chrome_options=options);
driver.get('http:/www.colfinancial.com.');
driver.implicitly_wait(10);

userfield1 = driver.find_element_by_name('txtUser1');
userfield1.send_keys(credentials['username1']);

userfield2 = driver.find_element_by_name('txtUser2');
userfield2.send_keys(credentials['username2']);

password = driver.find_element_by_name('txtPassword');
password.send_keys(credentials['password']);

# log-in to website
driver.find_element_by_xpath("//input[@type='button']").click();

driver.implicitly_wait(10);

# click trade > portfolio. Colfinancial uses iframe so we have to dynamically enter the iframe
driver.switch_to.frame(driver.find_element_by_id("headern"))
driver.find_element_by_id('CT').click()
driver.find_element_by_id('L1_3_4').click()

# jump out of iframe
driver.switch_to.default_content()
driver.implicitly_wait(20);

# enters iframe with the contents and tables
driver.switch_to.frame(driver.find_element_by_id("main"))
contents = driver.find_elements_by_tag_name('html')

# loads the contents into multiple dataframes, then closes the chrome session
all_tables = pd.read_html(contents[0].get_attribute('innerHTML')) # returns a list of dataframes
driver.close()


In [9]:
# This gets the list of equities from the PSE website from the fastquant module.
stock_table = fq.get_stock_table()
stock_table.head()

,Company Name,Stock Symbol,Sector,Subsector,Listing Date,company_id,security_id
0,"2GO Group, Inc.",2GO,Services,Transportation Services,"May 15, 1995",29,146
1,Asia Amalgamated Holdings Corporation,AAA,Holding Firms,Holding Firms,"Mar 22, 1973",55,347
2,"Atok-Big Wedge Co., Inc.",AB,Mining and Oil,Mining,"Jan 08, 1948",19,181
3,"AbaCore Capital Holdings, Inc.",ABA,Holding Firms,Holding Firms,"Oct 28, 1987",174,173
4,Asiabest Group International Inc.,ABG,Holding Firms,Holding Firms,"Jul 10, 1979",176,350


# Part 2: Data Preprocessing

##### In this section, we shall data preprocess the dataframes we have collected from the broker's website. First let us examine the data collected from the site.


In [10]:
# We have 7 tables that we can parse and clean through. Let's scan through it.
len(all_tables)

7

In [11]:
all_tables[2].head()

,0,1,2,3,4,5
0,Cash Balance,Cash Balance,Cash Balance,NaN,NaN,NaN
1,Actual Balance,Buying Power,NaN,NaN,Print Account Summary,View Monthly Account Ledger (current)
2,224796.24,224796.24,NaN,NaN,Print Monthly Account Ledger,View Monthly Account Ledger (historical)
3,NaN,NaN,NaN,NaN,View IPO Request Status,Activate | Deactivate E-Delivery of Confirma...
4,NaN,NaN,NaN,NaN,View Tender Offer,Transaction Invoice Daily | Historical


In [12]:
all_tables[3].head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,Stock Code:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Stock Code:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,EQUITIES Action Stock Code Stock Name Port...,EQUITIES Action Stock Code Stock Name Port...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,EQUITIES,EQUITIES,EQUITIES,EQUITIES,EQUITIES,EQUITIES,EQUITIES,EQUITIES,EQUITIES,EQUITIES,EQUITIES,NaN


##### After inspection of the tables, let us use the tables with index `2` and `3`. Table `2` contains the cash balance and Table `3` contains the equities. For now, I will disregard the mutual funds section.

##### 2.1 - To get the cash balance, let us just use the table with index `2`, and with additional slicing we can get my current cash amount.

In [13]:
# get cash balance 
cash_balance = all_tables[2].loc[1:2,0:1]
cash_balance.columns = cash_balance.loc[1]
cash_balance.drop(1, axis=0, inplace=True)

##### 2.2 - To get the equities and mutual fund balances, we shall use table with index `3`, and with additional slicing we can get my current equities and mutual fund positions.

In [14]:
# Here we shall take the 4th table from the list and drop all NAs. 
# Let's reset the index while doing this as well.
raw_table = all_tables[3].dropna(axis=0, how='all').dropna(axis=1, how='all')
raw_table = raw_table.reset_index().drop('index',axis=1)

In [15]:
# Because there can be N-number of stocks and N-number of mutual funds, 
# we have to slice through the raw table to prepare the data
display(raw_table.head(5))

,0,1,2,3,4,5,6,7,8,9,10
0,Stock Code:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Stock Code:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,EQUITIES Action Stock Code Stock Name Port...,EQUITIES Action Stock Code Stock Name Port...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,EQUITIES,EQUITIES,EQUITIES,EQUITIES,EQUITIES,EQUITIES,EQUITIES,EQUITIES,EQUITIES,EQUITIES,EQUITIES
4,Action,Stock Code,Stock Name,Portfolio %,Market Price,Average Price,Total Shares,Uncommitted Shares,Market Value,Gain / Loss,%Gain/ Loss


#### 

In [16]:
# In this scenario, we shall use `regex` to identify the cutoffs/indexes and the new headers
equities_index = raw_table[raw_table[0].str.match('^EQUITIES$') == True].index[0]
mutualfund_index = raw_table[raw_table[0].str.match('^MUTUAL FUNDS$') == True].index[0]

##### 2.2.a - Here we shall preprocess so that we will only have the equities in a proper dataframe

In [17]:
equities_balance = raw_table.loc[equities_index:mutualfund_index-3,:]
equities_balance.columns = equities_balance.loc[equities_index+1]
equities_balance.columns.name = 'idx'
equities_balance = (equities_balance
                    .drop([equities_index, equities_index+1],axis=0)
                    .reset_index()
                    .drop(['index', 'Action'],axis=1)
                    .dropna(axis=1, how='all')
                    .dropna(axis=0, how='all'))
if not equities_balance.empty:
    equities_balance['Class'] = "Equities"

In [18]:
stock_table = stock_table[['Company Name', 'Stock Symbol', 'Sector', 'Subsector']]

In [19]:
#This may remove any preferred shares in my equities account. For example, I have a `SMC2I` preffered share.

equities_balance_summary = (pd.merge(equities_balance, 
                                     stock_table, 
                                     left_on=['Stock Code'], 
                                     right_on=['Stock Symbol']))

In [20]:
# just to re-order the column names
equities_balance_summary = equities_balance_summary[['Company Name', 'Stock Symbol', 
                                                     'Sector', 'Subsector', 
                                                     'Portfolio %', 'Market Price', 
                                                     'Average Price', 'Total Shares', 
                                                     'Uncommitted Shares', 'Market Value',	
                                                     'Gain / Loss',	'%Gain/ Loss',	'Class']]
equities_balance_summary['Portfolio %'] = \
    (equities_balance_summary['Market Value'].astype(float) / 
     equities_balance_summary['Market Value'].astype(float).sum(axis=0))

equities_balance_summary['Portfolio %'] = equities_balance_summary['Portfolio %'].round(2)

today = datetime.datetime.now().strftime("%Y%m%d")
equities_balance_summary.to_csv(f"data/20230315/output/stock_{today}",index=False)



In [21]:
equities_balance_summary

,Company Name,Stock Symbol,Sector,Subsector,Portfolio %,Market Price,Average Price,Total Shares,Uncommitted Shares,Market Value,Gain / Loss,%Gain/ Loss,Class
0,"Aboitiz Equity Ventures, Inc.",AEV,Holding Firms,Holding Firms,0.05,49.3500,50.9499,2000,2000,97816.64,-4083.17,-4.01%,Equities
1,"AREIT, Inc.",AREIT,Property,Property,0.19,34.4500,32.8674,10500,10500,358487.56,13379.86,3.88%,Equities
2,Bank of the Philippine Islands,BPI,Financials,Banks,0.04,104.3000,101.8202,650,650,67188.23,1005.10,1.52%,Equities
3,"Century Pacific Food, Inc.",CNPF,Industrial,"Food, Beverage & Tobacco",0.05,24.5000,24.8971,4000,4000,97122.90,-2465.50,-2.48%,Equities
4,Converge Information and Communications Techno...,CNVRG,Services,Information Technology,0.04,15.0000,15.1855,4500,4500,66895.88,-1438.88,-2.11%,Equities
5,Citicore Energy REIT Corp.,CREIT,Property,Property,0.17,2.4500,2.1853,135000,135000,327789.79,32774.29,11.11%,Equities
6,"D&L Industries, Inc.",DNL,Industrial,"Food, Beverage & Tobacco",0.03,7.7700,7.9369,8500,8500,65453.90,-2009.75,-2.98%,Equities
7,First Philippine Holdings Corporation,FPH,Industrial,"Electricity, Energy, Power & Water",0.03,61.8500,62.7609,1000,1000,61296.44,-1464.46,-2.33%,Equities
8,"Globe Telecom, Inc.",GLO,Services,Telecommunications,0.01,1758.0000,1794.2790,10,10,17422.66,-520.13,-2.90%,Equities
9,"International Container Terminal Services, Inc.",ICT,Services,Transportation Services,0.03,197.7000,197.7637,300,300,58779.18,-549.93,-0.93%,Equities


##### Room for Improvements and TODOS:
1. Create a data pipeline that saves the `equities` and `cash` dataframes to a google repository (either google cloud storage or simple database). Create a scheduled script so this can be automated on the cloud.
2. Streamline data preparation and figure out if its possible to use the `fastquant` module for downloading individual stocks. Currently, not all of the stocks are listed or queryable. Currently, I will have to make do with downloading the historical prices from `https://ph.investing.com/`


##### Next Notebook
1. This was the data scraping notebook. In the next notebook, we shall explore basic portfolio analysis, such as:
<ul>
<li> Basic Descriptive Statistics
<li> Calculating Alpha and Beta
<li> Optimization of weights given existing portolio's positions (markowitz efficient frontier)
<li> Calculating Value At Risk
</ul>



